#### Backward generation(BWD)

1. generate data after integral

In [1]:
import random
from  random_unary_binary_tree import random_binary_trees
from  random_unary_binary_tree import traverse_unary_binary_prefix 
from infix_prefix import prefix_to_infix
from infix_prefix import infix_to_prefix
from sympy.parsing.sympy_parser import parse_expr
import time

In [2]:
from __future__ import division
from sympy import *
import numpy as np

In [3]:
x = Symbol('x', real=True)

In [4]:
def simplify_timeout(expr):
    from multiprocessing import Process, Manager
    
    result = None
    
    def f(d, expr):
        try:
            d['result'] = simplify(expr)
        except:
            d['result'] = "Error"

        
    with Manager() as manager:
        d = manager.dict()
        p = Process(target=f, args=(d, expr))
        p.start()
        p.join(timeout=3)
        if p.is_alive():
            p.terminate()
        else:    
            result = d["result"]
    if result==None:
        return expr
    else:
        return result

In [5]:
def parse_expr_timeout(string):
    from multiprocessing import Process, Manager
    
    expr = None
    
    def f(d, string):
        d['expr'] = parse_expr(string, local_dict={'x': x})

        
    with Manager() as manager:
        d = manager.dict()
        p = Process(target=f, args=(d, string))
        p.start()
        p.join(timeout=3)
        if p.is_alive():
            p.terminate()
        else:    
            expr = d["expr"]
    if expr==None or str(type(expr))=="<class 'sympy.calculus.util.AccumulationBounds'>":
        return None
    else:
        return expr

In [6]:
def generate_bwd(num):
    
    sequence = []

    start = time.time()

    i = 1
    while i<=num:
        n = random.randint(1, 15)
        root = random_binary_trees(n)
        result = traverse_unary_binary_prefix(root)
        if 'x' not in result:
            continue
        result_infix = prefix_to_infix(result) 
        result_expr = " ".join(result_infix)
        result_expr = parse_expr_timeout(result_expr)
        if result_expr==None:
            continue
        result_simp = simplify_timeout(result_expr)
        if result_simp == "Error":
            continue
        result_simp_prefix = infix_to_prefix(result_simp)
        if result_simp_prefix == False:
            continue
        try:
            expr_diff = diff(result_simp)
        except ValueError:
            continue
        expression = simplify_timeout(expr_diff)
        expression_prefix = infix_to_prefix(expression)
        if expression_prefix == False:
            continue  
        expression_prefix = " ".join(expression_prefix)
        result_simp_prefix = " ".join(result_simp_prefix)
        sequence.append(expression_prefix + "\t" + result_simp_prefix + "\n")
        i += 1

    end = time.time()
    print(end - start)
    return sequence

### Testing ...

In [8]:
from multiprocessing.pool import ThreadPool


start = time.time()
with ThreadPool(processes=8) as p:
    out = p.map(generate_bwd, [100]*2)
    p.terminate()
    p.join()
print("Finished: ", time.time()-start)    

57.6046929359436
77.08062887191772
Finished:  77.18485283851624


In [10]:
len(out)

2

In [13]:
file = open(r"./data/bwd_sample1000.txt", "w+")
for i in range(len(out)):
    sequence =out(i)
    file.writelines(sequence) 